# CS 419 Group Project

<h2>Developers:</h2>

- Ben Ortiz (benortiz)  
- Joshua Arleth (ja1057)   
- Anthony Guarino (ajg331) 
- Josh Zimmerman (jnz8)  
<br>

## 1 Problem Description

<h3>1.1 What is the goal of this project?</h3>

The goal of this project is to create a messaging app that allows it's users to send private messages to each other over the internet without worrying about their messages being read by potential attackers. This is achieved by using an encryption scheme similar to a vignere cipher, with some extra layers of security that prevent it to from being attacked by a Kasisky test. We do this by making words such as "the" to not be encrypted by into the same cipher text, and thus adding more security to messages in the app. We also try to make the key very hard to crack by giving each user a list of keys to use per message, thus giving another added layer of security. 



In [23]:
# TODO: Code Here if Needed, Else This Can Be Deleted

<h3>1.2 Where will this program be deployed or used?</h3>

The program is being designed for general consumers as the userbase. Whoever wants to send a message to another user on the app has access to use it. We hope to host this app in the future so it can be used on a web browser and anyone can log in as a user and start messaging each other with the goal of having the contents of their messages be secure. 

In [24]:
# TODO: Code Here if Needed, Else This Can Be Deleted

## 2 Problem Analysis
<h3>2.1 What is the threat model? Attackers’ knowledge, capability and goals?</h3>

 The threat model we assumed is an attacker using the Kasisky test to attempt to decipher the ciphertext into plaintext. The attackers knowledge is the ciphertext, their capability is their ability to intercept the ciphertext, and their goal is to turn the ciphertext into plaintext. Attackers may be able to receive sent messages that are encrypted, ciphertext, and possibly send their own plaintext messages, but not be able to see them after they have been encrypted back into ciphertext. 

In [25]:
# TODO: Code Here if Needed, Else This Can Be Deleted


<h3>2.2 What is the security goal and guarantee?</h3>

The security goal is to allow users to retain the privacy of their messages. The security is designed to protect the content of the messages, not necessarily the messages themselves. The guarantee is that the content of the messages will be encrypted enough that attackers will not be able to decrypt the messages from attacks such as a Kasisky test, as well as having the key being very difficult to crack.

In [26]:
# TODO: Code Here if Needed, Else This Can Be Deleted

## 3 Program Design

<h3>3.1 How did you achieve your goal? How is the functionality and security implemented?</h3>

We are designing an end to end encryption model that encrypts messages that are being sent and decrypts the messages once they have been confirmed as recieved by the destination. On delivery failure, the app will attempt to redeliver the message repeatedly until it recieves a proper ping or it times out. On timeout the message gets deleted and the user is notified of the deletion. The end implementation will be through a GUI app designed in Python that connects to an external server.

<h3>3.1.1 Encryption Algorithm Design</h3>

<h4>Basic Functionality</h4>

We designed and implemented our own encryption scheme which is built on top of the Vigenere Cipher. At the heart of the algorithm each character of the plain text is mapped to another random character according to a randomized key of fixed length. The new character is then inserted into the ciphertext and once all the characters in the plain text have been mapped and inserted into the ciphertext, the ciphertext is sent over the internet to the client. The client can then decrypt the ciphertext using the shared secret key and obtain the plaintext.

<h4>Levels of Complexity</h4>

Because there are known methods of cracking basic Viginere ciphers, such as the Kasisky Test, it is clear that we need to begin increasing the complexity of the above algorithm. Our encryption scheme accomplishes this in a variety of ways.

The first thing we did was increase the number of characters we can encrypt. Instead of only being able to encrypt 26 characters, like a basic Vigenere cipher, our algorithm can map 100 characters. Specifically our algorithm can map every single printable ascii character to any other printable ascii character. This significanlty increases the effectiveness of the algorithm as now spaces, tabs, newlines, numbers, and any other printable characters that could show up in the plaintext are now encrypted to some other printable character.

The second thing we did was randomize the keys. Each key is a fixed length sequence of random characters drawn from the set of printable ascii characters. The length of the keys will be set to the length of the maximum message size that we will allow for the messenger. This will make it so that for a single message the key never repeats. However an attacker intercepting multiple messages encrypted with the same key becomes a potential vulnerability. To mitigate this risk, each user has a set of keys. Each time a user sends a message the algorithm will randomely choose 1 of the keys to use for encryption. On the decryption side the client will need to decrypt the message with each key and then choose the result that isn't garbage text. To detect the non-grabage text the encryption algorithm inserts a check sum of the original plain text at the end of the encrypted message. The decrytper then extracts the check sum from the ciphertext, decrypts the ciphertext with each key, recalculates the check sum for each plain text, and returns the plain text whose check sum match the extracted check sum.

The third thing we did is gave the algorithm the ability to inject random characters into the ciphertext at every other index. This adds further randomness to the ciphertext and adds another layer to the complexity of the encryption scheme. The last thing the encryption algorithm does is insert random characters from the set of printable ascii characters into the ciphertext. The first thing the decrypter does after it extracts the check sum is remove every other character as it knows that these characters are not part of the message.


<h4>Further Improvements</h4>

These are some of the ideas for improvments to the algorithm that we will attempt to implement.

1:

Instead of inserting characters truly randomly, we insert characters with probabilities pulled from a probability distribution. This will give us the ability to mimic some level of language in the ciphertext. There are techniques for detecting true randomnes in the ciphertext thus by inserting characters with varying probabilities we can reduce the amount of randomness involved and make it much more difficult to detect.

2:

We rotate the set of keys used to encrypt the messages on a per message basis. For instance each user will have a list of 100 sets of keys. Each time a user sends a message it will use a different set of keys to encrypt. Thus in order for an attacker to obtain 2 messages encrypted with same set of keys he would need, on average, to intercept 100+ messages. 



<h3>3.1.2 Code Sample</h3>

In [27]:
#Below is the code for the encryption algo
#Sample output is shown below

import sys
import random
import string

#string of all printable characters
PRINTABLE_CHARS = string.printable

#debug variables to test various components
DEBUG_VIG = 1
DEBUG_RANDOM_KEY = 0
DEBUG_INSERT_RAND_LETTERS = 0
DEBUG_TEST_LETTER = 0
DEBUG_MAP_MSG = 0

#map string to values
def map_str_to_num(s):

    s_list = []
    s_idx = 0

    for x in s:
        if x in PRINTABLE_CHARS:
            s_idx = PRINTABLE_CHARS.find(x)
            s_list.append(s_idx) 
    return s_list

#sum elements of list and return mod 100
def compute_check_sum(m_list):
    c_sum = sum(m_list) % 100
    return c_sum

#grab last letter of msg and return value
def get_check_sum(m_list):
    c = m_list[-1:]
    return PRINTABLE_CHARS.find(c)

#map values to characters
def map_num_to_str(num_list):   
    s = ''
    for x in num_list:
        s += PRINTABLE_CHARS[x]     
    return s

#advanced encryption algorithm
#m - the message you want to encrypt
#k - a list of keys you want to use to encrypt m, the algo will select 1 at random
#insert_chars - whether or not you want to insert random letters every other character
def encrypt(m, k, insert_chars):

    c_list = []             #empty list for cipher values
    k_idx = 0               #index of the key letter in PRINTABLE_CHARS
    new_c_val = 0           #index of the new encrypted character
    k_list_len = len(k)     #numbers of keys in the key list
    ct = ''                 #empty output string

    #randomely choose a key from a list of keys passed in to encrypt with
    key_select = random.randint(0, k_list_len - 1)
    key = k[key_select]
    k_len = len(key)

    #map the key and msg to their indexes for encryption
    m_list = map_str_to_num(m)
    k_list = map_str_to_num(key)

    for x in m_list:
        new_c_val = (x + k_list[k_idx]) % len(PRINTABLE_CHARS)      #calculate new ciphertext value
        c_list.append(new_c_val)
        k_idx = (k_idx + 1) % k_len         #update location in key, use mod so if msg longer than key we loop back                   

    #map value list back to string
    ct = map_num_to_str(c_list)

    if insert_chars:
        ct = insert_rand_chars(ct)                              #make every other character a random letter

    #compute and insert check sum
    c_sum = compute_check_sum(m_list)
    ct += PRINTABLE_CHARS[c_sum]

    return ct

#advanced decryption algorithm
#inputs are the same as for encryption
def decrypt(m, keys, insert_chars):

    #get the checksum and remove checksum from ciphertext
    c_sum = get_check_sum(m)
    ct = m[:len(m) - 1]
    if insert_chars:
        ct = strip_rand_chars(m)      #remove every other character as we know they are garbage

    #need to decrypt for each key in list till the checksums match
    for k in keys:

        msg_list = []           
        k_idx = 0               
        new_c_val = 0           
        k_len = len(k)
        msg = ''

        m_list = map_str_to_num(ct)
        k_list = map_str_to_num(k)
        
        for x in m_list:

            new_c_val = ((x - k_list[k_idx]) + len(PRINTABLE_CHARS)) % len(PRINTABLE_CHARS)
            msg_list.append(new_c_val)

            k_idx = (k_idx + 1) % k_len     #do the opposite of encryption

        msg = map_num_to_str(msg_list)

        msg_c_sum = compute_check_sum(msg_list)     #get the checksum of the message we decrypted

        if msg_c_sum == c_sum:              
            return msg              #we found the right key

    return -1

#generate a random key
#n - the length of the key you want to generate
def get_rand_key(n):

    k = ''

    for __ in range(0, n):
        k += random.choice(string.printable)
    return k
    

#make every other character in a string a random character
def insert_rand_chars(m):

    new_list = []
    new_m = ""

    for c in m:
        new_char = random.choice(string.printable)
        new_list.append(c + new_char)

    new_m = ''.join(new_list)

    return new_m

#remove every other character from a string
def strip_rand_chars(m):

    split_string = [m[i:i+2] for i in range(0, len(m), 2)]
    for i in range(0, len(split_string)):
        split_string[i] = split_string[i][:-1]

    return ''.join(split_string)


def main():

    #this will test the encryption / decryption 
    if DEBUG_VIG == 1:
        
        #length of the key, using a small length for example
        n = 10
        
        #generate list of 2 keys for example
        key_list = [get_rand_key(n), get_rand_key(n)]

        m = 'hello my name is josh'
        ct = encrypt(m, key_list, True)
        pt = decrypt(ct, key_list, True)

        print(f'| Message: {m} |\n| Ciphertext: {ct} |\n| Plaintext: {pt} |')

        print(f'| Key: {key_list} |')

    print("Done!")

if __name__ == "__main__":
    main()

| Message: hello my name is josh |
TQg`5Uzxw |
| Plaintext: hello my name is josh |
| Key: [',!3pI=p6Y,', 'iPYo^nEA}:'] |
Done!


<h3>3.2 How can it provide the security guarantee you intended?</h3>

Our encryption scheme can provide the security we guarantee by basing our encrpytion scheme to a vigenere cipher, but with some extra layers of security, such as preventing it from being susceptible to a Kasisky test, thus making it very difficult to crack the ciphertext. 

In [28]:
# TODO: Code Here if Needed, Else This Can Be Deleted

## 4 Evaluation

<h3>4.1 Evaluation Strategy</h3>

What and how to evaluate? platform, datasets, user study?, how to measure, effectiveness, efficiency, security.  
  
What we will be evaluating is how difficult it is to crack the ciphertext from our encryption scheme. How we will evaluate the security of our instant messenger app is by first taking an example ciphertext and trying to decrypt it back into plaintext, and seeing how fast it can be done. We will do this by trying to attack it ourselves. 
  
We can evaluate the efficiency by calculating how long encryption / decryption takes for a variety of key and message lengths.

In [29]:
# TODO: Code Here if Needed, Else This Can Be Deleted

<h3>4.2 Experiments</h3>

How do you plan to do or design the experiments?  
  
We plan to do our experiments by first assuming the attacker can intercept the cipher text. Then attackers will attempt to crack the cipher text into plaintext. This will be done by us, and we will attempt to crack our own ciphertext back into plaintext, with the assumption the messages can already be intercepted. 

In [30]:
# TODO: Code Here if Needed, Else This Can Be Deleted

<h3>4.3 Results</h3>

What are the results?  
  
Assuming the results of our attacks are acceptable, we can expect to either, not be able to crack the ciphertext at all, crack some of the ciphertext, or actually crack the ciphertext completely. This is under the assumption that our attacks are successful, such as using the Kasisky test to attempt to crack the ciphertext, and finding the key. 

In [31]:
# TODO: Code Here if Needed, Else This Can Be Deleted

<h3>4.4 Analysis</h3>

What do these results mean?  
  
Assuming the results we get are either, we do not crack the ciphertext, only crack some of the ciphertext, or do not crack the ciphertext at all. The results can mean that our encryption scheme is good enough to not be broken by a Kasisky test assuming it wasn't able to be cracked. If the encryption scheme isn't good enough, where some of the ciphertext can be decrpyted or not decrypted at all, then we know that our encrpytion scheme is strong enough to not be cracked by the Kasisky test, and is sufficient enough in providing the security we need for our messaing app. 

In [32]:
# TODO: Code Here if Needed, Else This Can Be Deleted

## 5 Process Description
Overall timeline.
Where are we now?   
  
Currently the software can connect to a specific port on a server and send a response to the client. It will also be able to send simple messages through a basic client. We have also created an encryption scheme that will encrypt the messages that is sent between 2 messengers. The timeline in the future is to complete the encrpytion scheme to make sure we are satisfied with how secure it is, create the messaging app itself that allows users to send message, create a GUI for the messaging app, host it, and merge it all together to make sure it works.

In [33]:
# TODO: Code Here if Needed, Else This Can Be Deleted

## 6 Group and Artifacts
Group members and each of their contributions in %.   
- Ben Ortiz (benortiz)  -  25%  
- Joshua Arleth (ja1057) - 25%  
- Anthony Guarino (ajg331) - 25%  
- Josh Zimmerman (jnz8) - 25%  

Project repository address.   
https://github.com/Ben-Ortiz/cs419_group_project  
  
What is proud part of this project, for each of you?  
- Ben Ortiz (benortiz)  
- Joshua Arleth (ja1057)  
- Anthony Guarino (ajg331)  
- Josh Zimmerman (jnz8)  

In [34]:
# TODO: Code Here if Needed, Else This Can Be Deleted


## 7 Questions to Answer
When do you want us to freeze the repository?  
  
This does not apply to us yet. 